In [420]:
import selenium as sel
from selenium.webdriver.common.by import By
import pandas as pd
import time
import datetime
import base as bs
import ElTiempoFunciones as etf

import os as os
import csv
import json

Librerias que puedo mirar luego en detalle

from selenium.webdriver.common.by import By


In [421]:
f = open('config.json')
empresas = json.load(f)

if not os.path.exists('../data/raw/eltiempo.csv'):
    os.makedirs('../data/raw/eltiempo.csv')
    
#for i in empresas['LaRepublica']:
    

In [422]:
#Empresa con la cual vamos a extraer los articulos
# TODO Tener una lista que itere por todos lo clientes
empresa = input("Digite la empresa a extraer: ")

In [423]:
#cerar driver... MODIFICAR DEPENDIENDO DEL NAVEGADOR
driver = sel.webdriver.Edge()
driver.get(f'https://www.eltiempo.com/buscar?q={empresa}')
# time.sleep(2)
driver.implicitly_wait(10) #Nueva metodología de wait

In [424]:
# Buscar el class que abarca a toda la página 
# buscar = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]')
# articulos = buscar.find_elements(By.CLASS_NAME,"listing")

In [425]:
# El número total de ventanas que tiene la búsqueda

# total_num_ventanas = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]/div[2]/div[2]/div/div[11]/div/ul/li[10]/a').get_attribute('href')
# print(total_num_ventanas)

https://www.eltiempo.com/buscar/2766?q=ecopetrol


In [426]:
total_num_ventanas = f'https://www.eltiempo.com/buscar/20?q=ecopetrol' # Solo para el test

In [427]:
titulares=[]
get_url = driver.current_url
pag_actual = 1


while(get_url != total_num_ventanas):
        buscar = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]')
        articulos = buscar.find_elements(By.CLASS_NAME,"listing")
        for articulos in articulos:
                aux = articulos.find_element(By.XPATH, './/h3[contains(@class, "title-container")]')
                url = aux.find_element(By.XPATH,'.//a').get_attribute('href')
                # print(url)

                if not(bs.existedb(url, "eltiempo")):

                        titulo = articulos.find_element(By.CLASS_NAME,"title-container").text
                        # print(titulo)
                        resumen = articulos.find_element(By.CLASS_NAME,"epigraph-container").text
                        # print(resumen)
                        fechaPub = articulos.find_element(By.CLASS_NAME,"published-at").text
                        # print(fechaPub)
                        tema = articulos.find_element(By.CLASS_NAME,"category").text
                        # print(tema)
                        

                        titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                                        'Titulo':titulo,
                                        'Fecha Publicacion':fechaPub,
                                        'Tema':tema,
                                        'URL':url,
                                        'Resumen':resumen,
                                        'Empresa':empresa})
        pag_actual=pag_actual+1
        lik = driver.get(f'https://www.eltiempo.com/buscar/{pag_actual}?q=ecopetrol')
        driver.implicitly_wait(10) #Nueva metodología de wait
        get_url = driver.current_url
# driver.quit()
# print(titulares)

In [428]:
buscar = driver.find_element(By.XPATH,'//*[@id="main-container"]/div[16]/div[2]')
articulos = buscar.find_elements(By.CLASS_NAME,"listing")
for articulos in articulos:
        titulo = articulos.find_element(By.CLASS_NAME,"title-container").text
        # print(titulo)
        resumen = articulos.find_element(By.CLASS_NAME,"epigraph-container").text
        # print(resumen)
        fechaPub = articulos.find_element(By.CLASS_NAME,"published-at").text
        # print(fechaPub)
        tema = articulos.find_element(By.CLASS_NAME,"category").text
        # print(tema)
        aux = articulos.find_element(By.XPATH, './/h3[contains(@class, "title-container")]')
        url = aux.find_element(By.XPATH,'.//a').get_attribute('href')
        # print(url)

        titulares.append({'Fecha Extraccion':datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                        'Titulo':titulo,
                        'Fecha Publicacion':fechaPub,
                        'Tema':tema,
                        'URL':url,
                        'Resumen':resumen,
                        'Empresa':empresa})

In [429]:
columnas = ['Fecha Extraccion','Titulo', 'Fecha Publicacion','Resumen','URL','Empresa','Tema']
# Para AGREGAR titulares a un archivo.
with open('../data/raw/eltiempo.csv', 'a', newline='', errors='ignore') as csv_file:
    dict_object = csv.DictWriter(csv_file, fieldnames=columnas) 
  
    dict_object.writerows(titulares)

#Para crear un NUEVO archivo con los titulares
pd.json_normalize(titulares).to_csv(f'../data/raw/eltiempo.csv',index=False, encoding='latin-1', errors='ignore', columns=columnas)